In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.


In [3]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

In [4]:
train_df = pd.read_csv('../input/train.csv')
test_df = pd.read_csv('../input/test.csv')
df = [train_df, test_df]

In [5]:
train_df.head()

In [6]:
train_df.info()
test_df.info()

In [7]:
train_df.describe(include=['O'])

In [8]:
train_df[['Pclass','Survived']].groupby('Pclass').mean()

In [9]:
grid = sns.FacetGrid(train_df,col='Survived')
grid.map(plt.hist,'Age',bins=20)

In [10]:
grid = sns.FacetGrid(train_df, row='Embarked', size=2.2, aspect=1.6)
grid.map(sns.pointplot, 'Pclass', 'Survived', 'Sex', palette='deep')
grid.add_legend()

In [11]:
train_df = train_df.drop(['Ticket','Cabin'],axis=1)
test_df = test_df.drop(['Ticket','Cabin'],axis=1)
combine = [train_df,test_df]

In [12]:
for dataset in combine:
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)

pd.crosstab(train_df['Title'],train_df['Sex'])

In [13]:
for dataset in combine:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col',\
 	'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Other')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
    
train_df[['Title', 'Survived']].groupby(['Title']).mean()

In [14]:
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Other": 5}
for dataset in combine:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)

train_df.head()

In [15]:
train_df = train_df.drop(['Name', 'PassengerId'], axis=1)
test_df = test_df.drop(['Name'], axis=1)
combine = [train_df, test_df]

In [16]:
for dataset in combine:
    dataset['Sex'] = dataset['Sex'].map( {'female': 1, 'male': 0} ).astype(int)

train_df.head()

In [17]:
age_median = np.zeros((2,3))
for dataset in combine:
    for sex in range(0,2):
        for pclass in range (0,3):
            age_df = dataset[(dataset['Sex']==sex) & (dataset['Pclass']==pclass+1)]['Age'].dropna()
            age_median[sex,pclass] = int(age_df.median())
    for sex in range(0,2):
        for pclass in range (0,3):
            dataset.loc[(dataset.Age.isnull()) & (dataset['Sex']==sex) & (dataset['Pclass']==pclass+1),'Age'] = age_median[sex,pclass]


In [18]:
bins = (0,6,12,18,25,35,60,100)
groups = [0,1,2,3,4,5,6]
for dataset in combine:
    dataset.Age = pd.cut(dataset['Age'],bins,labels=groups).astype(int)

In [19]:
for dataset in combine:
    dataset['isAlone'] = 0
    dataset.loc[(dataset['SibSp']==0) & (dataset['Parch']==0), 'isAlone'] = 1

train_df = train_df.drop(['SibSp','Parch'],axis=1)
test_df = test_df.drop(['SibSp','Parch'],axis=1)
combine = [train_df, test_df]

In [20]:
port = train_df.Embarked.dropna().mode()[0]
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].fillna(port)

In [21]:
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)

In [22]:
test_df['Fare'].fillna(test_df['Fare'].dropna().median(),inplace=True)
test_df.info()

In [23]:
fare_band = pd.qcut(train_df['Fare'],5)
fare_band
bins = (-0.05,7.854,10.5,21.679,39.688,512.33)
groups = (0,1,2,3,4)
for dataset in combine:
    dataset.Fare = pd.cut(dataset['Fare'],bins,labels=groups).astype(int)

In [24]:
X_train = train_df.drop("Survived", axis=1)
Y_train = train_df["Survived"]
X_test  = test_df.drop("PassengerId", axis=1)

In [25]:
logreg = LogisticRegression()
logreg.fit(X_train, Y_train)
Y_test = logreg.predict(X_test)
acc_log = round(logreg.score(X_train, Y_train) * 100, 2)
acc_log

In [26]:
coeff_df = pd.DataFrame()
coeff_df['Feature'] = train_df.columns.delete('0')
coeff_df["Correlation"] = pd.Series(logreg.coef_[0])
coeff_df.sort_values(by='Correlation', ascending=False)

In [27]:
svc = SVC()
svc.fit(X_train, Y_train)
Y_pred = svc.predict(X_test)
acc_svc = round(svc.score(X_train, Y_train) * 100, 2)
acc_svc

In [28]:
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, Y_train)
Y_pred = knn.predict(X_test)
acc_knn = round(knn.score(X_train, Y_train) * 100, 2)
acc_knn

In [29]:
# Decision Tree

decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)
Y_pred = decision_tree.predict(X_test)
acc_decision_tree = round(decision_tree.score(X_train, Y_train) * 100, 2)
acc_decision_tree

In [30]:
# Random Forest

random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, Y_train)
acc_random_forest = round(random_forest.score(X_train, Y_train) * 100, 2)
acc_random_forest

In [ ]:
submission = pd.DataFrame({
        "PassengerId": test_df["PassengerId"],
        "Survived": Y_pred
    })
submission.to_csv('submission.csv', index=False)